In [61]:
import requests
import pandas as pd
import ta
import time
import numpy as np
from datetime import datetime, timedelta

# === CONFIGURATION ===
api_key = 'f5754d3325dc4149ba98663c7dd9821e'
symbol = 'XAU/USD'
interval = '5min'
start_date = '2024-01-01'
end_date = '2024-06-30'

# === Téléchargement ===
def fetch_data_page(start_date, end_date):
    url = 'https://api.twelvedata.com/time_series'
    params = {
        'symbol': symbol,
        'interval': interval,
        'start_date': start_date,
        'end_date': end_date,
        'apikey': api_key,
        'format': 'JSON',
        'order': 'ASC',
        'timezone': 'UTC',
        'outputsize': 5000
    }
    response = requests.get(url, params=params)
    data = response.json()
    if 'values' in data:
        return pd.DataFrame(data['values'])
    else:
        print("Erreur:", data)
        return None

# === Boucle pour historique ===
all_data = pd.DataFrame()
current_start = datetime.strptime(start_date, "%Y-%m-%d")
final_end = datetime.strptime(end_date, "%Y-%m-%d")

print("📥 Téléchargement des données en 5min par batch...")

while current_start < final_end:
    current_end = current_start + timedelta(days=10)
    if current_end > final_end:
        current_end = final_end

    print(f" - {current_start.date()} -> {current_end.date()}")
    df_page = fetch_data_page(current_start.strftime("%Y-%m-%d"), current_end.strftime("%Y-%m-%d"))
    if df_page is not None and not df_page.empty:
        all_data = pd.concat([all_data, df_page])

    current_start = current_end + timedelta(days=1)
    time.sleep(10)

# === Nettoyage ===
if all_data.empty:
    print("❌ ERREUR : aucune donnée téléchargée.")
    exit()

print("✅ Données téléchargées. Nettoyage...")
all_data['datetime'] = pd.to_datetime(all_data['datetime'])
all_data = all_data.sort_values('datetime').reset_index(drop=True)

# Colonnes numériques
for col in ['open', 'high', 'low', 'close']:
    all_data[col] = all_data[col].astype(float)

# === Indicateurs techniques ===
print("✅ Calcul des indicateurs techniques...")

close_series = all_data['close']
high_series = all_data['high']
low_series = all_data['low']

# RSI
all_data['rsi'] = ta.momentum.RSIIndicator(close_series, window=10).rsi()

# EMA9 & EMA21
all_data['ema_9'] = close_series.ewm(span=9, adjust=False).mean()
all_data['ema_21'] = close_series.ewm(span=21, adjust=False).mean()

# MACD
macd_calc = ta.trend.MACD(close_series)
all_data['macd_line'] = macd_calc.macd()

# ✅ ➜ Distance EMA
all_data['ema_distance'] = abs(all_data['ema_9'] - all_data['ema_21'])

# ✅ ➜ ATR (Average True Range)
atr_indicator = ta.volatility.AverageTrueRange(high_series, low_series, close_series, window=14)
all_data['atr'] = atr_indicator.average_true_range()


# ✅ ➜ Volatilité locale : std des closes sur 10 périodes
all_data['volatility_close_std'] = close_series.rolling(window=10).std()

# ✅ ➜ Pente EMA (slope)
all_data['ema_9_slope'] = all_data['ema_9'].diff(3)
all_data['ema_21_slope'] = all_data['ema_21'].diff(3)

# === Logique croisement EMA actuel
all_data['ema9_above_ema21'] = all_data['ema_9'] > all_data['ema_21']

# === Target : prédire croisement dans N bougies
print("✅ Calcul de la target pour prédire le CROISEMENT futur...")
future_horizon = 2
all_data['future_ema9_above'] = all_data['ema9_above_ema21'].shift(-future_horizon)

all_data['target'] = 0
all_data.loc[
    (all_data['ema9_above_ema21'] == False) & (all_data['future_ema9_above'] == True),
    'target'
] = 1
all_data.loc[
    (all_data['ema9_above_ema21'] == True) & (all_data['future_ema9_above'] == False),
    'target'
] = -1

# === Nettoyage final
all_data = all_data.dropna()

print(all_data.head())

# === Sauvegarde CSV ===
all_data.to_csv('gold_features_ema_cross_prediction_enriched.csv', index=False)
print("✅ Données sauvegardées dans gold_features_ema_cross_prediction_enriched.csv")


📥 Téléchargement des données en 5min par batch...
 - 2024-01-01 -> 2024-01-11
 - 2024-01-12 -> 2024-01-22
 - 2024-01-23 -> 2024-02-02
 - 2024-02-03 -> 2024-02-13
 - 2024-02-14 -> 2024-02-24
 - 2024-02-25 -> 2024-03-06
 - 2024-03-07 -> 2024-03-17
 - 2024-03-18 -> 2024-03-28
 - 2024-03-29 -> 2024-04-08
 - 2024-04-09 -> 2024-04-19
 - 2024-04-20 -> 2024-04-30
 - 2024-05-01 -> 2024-05-11
 - 2024-05-12 -> 2024-05-22
 - 2024-05-23 -> 2024-06-02
 - 2024-06-03 -> 2024-06-13
 - 2024-06-14 -> 2024-06-24
 - 2024-06-25 -> 2024-06-30
✅ Données téléchargées. Nettoyage...
✅ Calcul des indicateurs techniques...
✅ Calcul de la target pour prédire le CROISEMENT futur...
              datetime        open         high         low       close  \
25 2024-01-02 01:05:00  2066.98999  2069.739990  2066.98999  2067.61011   
26 2024-01-02 01:10:00  2067.55005  2069.610110  2067.40991  2069.34009   
27 2024-01-02 01:15:00  2069.46997  2069.889890  2067.73999  2068.34009   
28 2024-01-02 01:20:00  2068.34009  2069

In [62]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight

# === 1️⃣ Chargement des données ===
df = pd.read_csv('gold_features_ema_cross_prediction_enriched.csv')
print("✅ Données chargées")
print(df.head())

# === 2️⃣ Sélection des features ===
features = [
    'rsi',
    'ema_9', 'ema_21', 'ema_distance',
    'macd_line',
    'atr',
    'volatility_close_std',
    'ema_9_slope', 'ema_21_slope'
]

X = df[features]
y = df['target']

# === 3️⃣ Calcul des poids des classes ===
cw = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = {cls: weight for cls, weight in zip(np.unique(y), cw)}
print("✅ Class Weights:", class_weights)

# === 4️⃣ Split des données ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
print("✅ Split terminé")

# === 5️⃣ Entraînement du modèle ===
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight=class_weights,
    max_depth=12,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("✅ Modèle entraîné")

# === 6️⃣ Évaluation ===
y_pred = model.predict(X_test)
print("\n✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))

# === 7️⃣ Sauvegarde du modèle ===
with open('gold_cross_prediction_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("✅ Modèle sauvegardé sous gold_cross_prediction_model.pkl")



✅ Données chargées
              datetime        open         high         low       close  \
0  2024-01-02 01:05:00  2066.98999  2069.739990  2066.98999  2067.61011   
1  2024-01-02 01:10:00  2067.55005  2069.610110  2067.40991  2069.34009   
2  2024-01-02 01:15:00  2069.46997  2069.889890  2067.73999  2068.34009   
3  2024-01-02 01:20:00  2068.34009  2069.010010  2067.42993  2068.73999   
4  2024-01-02 01:25:00  2068.55005  2069.070068  2068.32007  2068.62012   

         rsi        ema_9       ema_21  macd_line  ema_distance       atr  \
0  74.858231  2065.766109  2065.168574   0.382136      0.597536  1.443656   
1  81.484355  2066.480906  2065.547802   0.660521      0.933103  1.497695   
2  69.688226  2066.852742  2065.801647   0.791329      1.051096  1.544281   
3  71.520167  2067.230192  2066.068769   0.916698      1.161423  1.546838   
4  70.108962  2067.508178  2066.300710   0.994912      1.207468  1.489921   

   volatility_close_std  ema_9_slope  ema_21_slope  ema9_above_ema2